# Install AG2 + Materials Project dependencies

In [17]:
!pip install "ag2[openai]" -q


import sys
!{sys.executable} -m pip install pymatgen mp-api numpy cython -q


!pip install ipywidgets jupyterlab_widgets -q

print("✓ AG2 + Pymatgen + MP-API installed.")


✓ AG2 + Pymatgen + MP-API installed.


# Imports + LLM Configuration


In [18]:
# Standard library imports
import os
import json
import random
from typing import Annotated, Any
from datetime import datetime, timedelta

# AG2 core imports
from autogen import ConversableAgent, LLMConfig
from autogen.agentchat.group import ContextVariables
from autogen.tools import tool
from autogen.agentchat.groupchat import GroupChat, GroupChatManager
from autogen.agentchat import ReplyResult

# Tools + External APIs
from mp_api.client import MPRester

# Coding tools (for future coding agent)
from autogen.coding.local_commandline_code_executor import LocalCommandLineCodeExecutor
from autogen.coding.base import CodeBlock
from pydantic import BaseModel, Field

# Configure the LLM
llm_config = LLMConfig(config_list={
    "api_type": "openai",
    "model": "gpt-4o",
    "api_key": os.environ["OPENAI_API_KEY"],
})

print("✓ Imports loaded and LLM configured.")


✓ Imports loaded and LLM configured.


# User query + Context Variables

In [30]:
from autogen.agentchat.group import ContextVariables

# Development mode switch
DEV_MODE = False   # Set to False to enable real user input

if DEV_MODE:
    # No input while developing
    user_query = "test query"
else:
    # Real interactive input (final version)
    user_query = input("Enter your materials design challenge: ")

# Initialize context variables
context_variables = ContextVariables(
    data={
        "user_query": user_query,
        "explained_terms": None,
        "mp_query": None,
        "mp_results": None,
        "final_conclusion": None,
        "next_agent": "AgentA_Explainer"
    }
)

print("✓ Context variables initialized.")



Enter your materials design challenge:  hello


✓ Context variables initialized.


# TOOLS (A → B → C → D)

In [27]:

# Tool A — explain_query
@tool(name="explain_query", description="Explain the user's materials-science query.")
def explain_query_tool(query: str, context_variables: dict):
    if not isinstance(query, str):
        raise ValueError("The 'query' parameter must be a string.")

    if len(query.strip()) == 0:
        raise ValueError("Query cannot be empty.")

    explanation = (
        f"Explanation of user query:\n\n"
        f"- Original query: '{query}'\n"
        f"- This query requests information about material properties.\n"
        f"- Agent A will extract key terms and concepts.\n"
    )

    context_variables["explained_terms"] = explanation

    return {
        "message": explanation,
        "next_agent": "AgentB_MaterialsRetriever",
        "context_variables": context_variables
    }



# Tool B — materials_project_query
@tool(name="materials_project_query",
      description="Query the Materials Project database with validation.")
def materials_project_query_tool(query: dict, context_variables: dict):

    api_key = os.getenv("MP_API_KEY")
    if not api_key:
        raise ValueError("Materials Project API key not found. Set MP_API_KEY.")

    # --- Validation Checks ---
    if not isinstance(query, dict):
        raise ValueError("Query must be a dictionary.")

    if "criteria" not in query:
        raise ValueError("Missing required field: 'criteria'.")

    if "properties" not in query:
        raise ValueError("Missing required field: 'properties'.")

    criteria = query["criteria"]
    properties = query["properties"]

    if not isinstance(criteria, dict):
        raise ValueError("'criteria' must be a dictionary.")

    if not isinstance(properties, list):
        raise ValueError("'properties' must be a list.")

    if len(properties) == 0:
        raise ValueError("Properties list cannot be empty.")

    # Optional: limit allowed fields for safety
    allowed_numeric_filters = {"band_gap", "density", "e_above_hull"}

    for key, value in criteria.items():
        if key not in allowed_numeric_filters:
            raise ValueError(f"Invalid filter '{key}'. Allowed filters: {allowed_numeric_filters}")

        if not isinstance(value, dict):
            raise ValueError("Each filter value must be a dictionary like {'$gt': number}.")

        for op, val in value.items():
            if op not in ["$gt", "$lt", "$gte", "$lte"]:
                raise ValueError(f"Invalid operator '{op}'. Use one of $gt, $lt, $gte, $lte.")

            if not isinstance(val, (int, float)):
                raise ValueError("Numeric filter values must be numbers.")


    # --- Execute MP query safely ---
    with MPRester(api_key) as mpr:
        results = mpr.query(criteria=criteria, properties=properties)

    context_variables["mp_query"] = query
    context_variables["mp_results"] = results

    return {
        "message": f"Retrieved {len(results)} materials.",
        "next_agent": "AgentC_Analyzer",
        "context_variables": context_variables
    }



# Tool C — final_conclusion
@tool(name="final_conclusion", description="Produce a structured, multi-dimensional analysis of retrieved materials.")
def final_conclusion_tool(mp_results: list, context_variables: dict):

  

    if not isinstance(mp_results, list):
        raise ValueError("mp_results must be a list.")

    if len(mp_results) == 0:
        raise ValueError("mp_results is empty. No materials to analyze.")


    structured_output = []

    for entry in mp_results:

        material_block = {
            "Material": {
                "ID": entry.get("material_id", None),
                "Formula": entry.get("pretty_formula", None),
                "Bandgap": entry.get("band_gap", None),
                "Density": entry.get("density", None),
                "Volume": entry.get("volume", None),
                "Energy_above_hull": entry.get("e_above_hull", None),
            },

            # --- NEW SECTIONS (per mentor request) ---
            "Usage": {
                "Applications": "Potential applications based on retrieved properties.",
                "Industry_relevance": "Why this material could matter for industry.",
            },

            "Sustainability": {
                "Abundance": "Placeholder for abundance / criticality analysis.",
                "Environmental_impact": "Placeholder evaluation of toxicity / extraction cost.",
                "Recyclability": "Placeholder for recyclability commentary.",
            },

            "Weaknesses": {
                "Structural_limitations": "Placeholder for mechanical or thermal limitations.",
                "Economic_limitations": "Placeholder for cost or manufacturability issues.",
            }
        }

        structured_output.append(material_block)


    # Save to context variables
    context_variables["final_conclusion"] = structured_output

    return {
        "message": structured_output,
        "next_agent": "Human",      # The analysis ends; return to the user.
        "context_variables": context_variables
    }


# Tool D — python_code_block
project_folder = os.path.abspath("ag2_project")
os.makedirs(project_folder, exist_ok=True)

@tool(name="python_code_block", description="Execute Python code and save to file.")
def python_code_block_tool(code: str, file_name: str, context_variables: dict):

    if not isinstance(code, str):
        raise ValueError("Code must be a string.")

    if not isinstance(file_name, str):
        raise ValueError("file_name must be a string.")

    result = python_code_block(
        code=code,
        code_file_name=file_name,
        context_variables=context_variables
    )

    return {
        "message": result.message,
        "next_agent": None,
        "context_variables": context_variables

    }


# Tool E — save_code_to_file
@tool(name="save_code_to_file", description="Save a Python code string to a file inside the project folder.")
def save_code_to_file_tool(code: str, file_name: str, context_variables: dict):

    # Basic validation
    if not isinstance(code, str):
        raise ValueError("The 'code' parameter must be a string.")

    if not isinstance(file_name, str):
        raise ValueError("The 'file_name' parameter must be a string.")

    if len(file_name.strip()) == 0:
        raise ValueError("file_name cannot be empty.")

    # Compute the path
    file_path = os.path.join(project_folder, file_name)

    # Ensure folder exists
    os.makedirs(os.path.dirname(file_path), exist_ok=True)

    # Write the file
    with open(file_path, "w") as f:
        f.write(code)

    message = f"Code saved to: {file_path}"

    return {
        "message": message,
        "next_agent": None,
        "context_variables": context_variables
    }

    


# Manual tests for Materials Project tool


In [ ]:

print("Running manual tests for materials_project_query_tool...\n")

# Test 1 — Simple band gap query
try:
    test_query_1 = {
        "criteria": {"band_gap": {"$gt": 3}},
        "properties": ["material_id", "pretty_formula", "band_gap"]
    }
    result_1 = materials_project_query_tool(test_query_1, {})
    print("Test 1 passed:", result_1["message"])
except Exception as e:
    print("Test 1 failed:", e)


# Test 2 — Invalid operator (should raise ValueError)
try:
    test_query_2 = {
        "criteria": {"band_gap": {"INVALID_OP": 3}},
        "properties": ["material_id"]
    }
    result_2 = materials_project_query_tool(test_query_2, {})
    print("Test 2 FAILED (should have raised error)")
except Exception as e:
    print("Test 2 passed (expected error):", e)


# Test 3 — Missing properties
try:
    test_query_3 = {
        "criteria": {"band_gap": {"$gt": 2}}
    }
    result_3 = materials_project_query_tool(test_query_3, {})
    print("Test 3 FAILED (should have raised error)")
except Exception as e:
    print("Test 3 passed (expected error):", e)


# Test 4 — Empty properties list
try:
    test_query_4 = {
        "criteria": {"density": {"$lt": 5}},
        "properties": []
    }
    result_4 = materials_project_query_tool(test_query_4, {})
    print("Test 4 FAILED (should have raised error)")
except Exception as e:
    print("Test 4 passed (expected error):", e)


# Test 5 — Invalid filter key
try:
    test_query_5 = {
        "criteria": {"invalid_property": {"$gt": 3}},
        "properties": ["material_id"]
    }
    result_5 = materials_project_query_tool(test_query_5, {})
    print("Test 5 FAILED (should have raised error)")
except Exception as e:
    print("Test 5 passed (expected error):", e)


print("\nManual tool tests completed.")


# AGENTS (A → B → C → D + HUMAN)

In [21]:
# --- Agent A: Explainer ---

explainer_message = """
You are AgentA_Explainer.

### ROLE
You interpret the user's materials-science question and produce a structured scientific explanation.

### YOUR TASKS
- Parse the user's query from context_variables["user_query"].
- Identify relevant scientific terminology (e.g., bandgap, symmetry, density, conductivity).
- Generate a clear explanation of what the user is asking.
- Clarify key terms in simple but technically correct language.

### TOOL YOU MUST USE
- You MUST call the tool: explain_query
- Never produce free text.
- The explanation MUST be generated ONLY through explain_query.

### OUTPUT FORMAT REQUIRED
The tool output should follow this structure:
{
  "explained_terms": "<detailed scientific explanation>",
  "next_agent": "AgentB_MaterialsRetriever"
}

### FORBIDDEN ACTIONS
- Do NOT solve the materials-science query.
- Do NOT retrieve materials.
- Do NOT analyze data.
- Do NOT talk without using the tool.
"""

AgentA_Explainer = ConversableAgent(
    name="AgentA_Explainer",
    llm_config=llm_config,
    system_message=explainer_message,
    human_input_mode="NEVER",
    functions=[explain_query_tool],   
)



# --- Agent B — Materials Retriever ---

retriever_message = """
You are AgentB_MaterialsRetriever.

### ROLE
You transform the explained query into a valid Materials Project query.

### YOUR TASKS
- Read the explanation from context_variables["explained_terms"].
- Construct a valid Materials Project API search request using criteria + properties.
- Validate the query structure before calling the tool.
- Use the tool to fetch materials.

### TOOL YOU MUST USE
- materials_project_query
- You cannot talk without calling the tool.

### MUST VALIDATE BEFORE TOOL CALL
- Ensure criteria is a dict.
- Ensure properties is a list.
- Raise ValueError for invalid keys:
  - Allowed numeric filters: band_gap, density, formation_energy_per_atom
  - Allowed element filters: elements, chemsys, formula
- Raise ValueError for missing or empty fields.

### OUTPUT FORMAT REQUIRED
{
  "mp_query": {...},
  "mp_results": [...],
  "next_agent": "AgentC_Analyzer"
}

### FORBIDDEN ACTIONS
- No free text.
- No analysis (belongs to Agent C).
- No coding (belongs to Agent D).
"""

AgentB_MaterialsRetriever = ConversableAgent(
    name="AgentB_MaterialsRetriever",
    llm_config=llm_config,
    system_message=retriever_message,
    human_input_mode="NEVER",
    functions=[materials_project_query_tool],   
)



# --- Agent C — Analyzer ---

analyzer_message = """
You are AgentC_Analyzer.

### ROLE
You analyze retrieved Materials Project results stored in context_variables["mp_results"].

### YOUR TASKS
- Read MP results and extract chemical and physical information.
- Organize material data into a highly structured scientific summary.
- Produce a final human-readable conclusion via the tool final_conclusion.

### REQUIRED OUTPUT STRUCTURE
For each material produce:

Material:
- ID:
- Formula:
- Bandgap:
- Density:
Usage:
Sustainability:
Weaknesses:

### TOOL YOU MUST USE
- final_conclusion
- You MUST produce all output using the tool.
- No direct text output.

### FORBIDDEN ACTIONS
- Do NOT retrieve materials (belongs to Agent B)
- Do NOT explain user queries (belongs to Agent A)
- Do NOT execute Python (belongs to Agent D)
"""

AgentC_Analyzer = ConversableAgent(
    name="AgentC_Analyzer",
    llm_config=llm_config,
    system_message=analyzer_message,
    human_input_mode="NEVER",
    functions=[final_conclusion_tool],     



# --- Agent D — Coding Agent ---

coder_message = """
You are AgentD_Coder.

### ROLE
Generate and execute Python code when requested by Agent C.

### YOUR TASKS
- Write Python functions, calculations, or plots.
- Execute Python code strictly through the python_code_block tool.
- Save executed code into ag2_project/<file_name>.
- Return only clean results.

### TOOL YOU MUST USE
- python_code_block

### REQUIRED OUTPUT FORMAT
{
  "message": "<execution output>",
  "next_agent": null
}

### FORBIDDEN ACTIONS
- Never produce plain Python code in the chat.
- Never answer scientific questions.
- Never retrieve materials.
"""

AgentD_Coder = ConversableAgent(
    name="AgentD_Coder",
    llm_config=llm_config,
    system_message=coder_message,
    human_input_mode="NEVER",
    functions=[python_code_block_tool]   # Already correct
)



# --- Human Agent ---

Human = ConversableAgent(
    name="Human",
    human_input_mode="ALWAYS"
)


# Pattern definition 


In [31]:
# --- Pattern: next speaker logic ---

def select_next_speaker(last_speaker, groupchat):
    ctx = groupchat.context_variables.data
    na = ctx.get("next_agent")

    # If a tool explicitly sets the next agent:
    if na:
        ctx["next_agent"] = None  # reset after using it
        return groupchat.agent_lookup[na]

    # First turn → Human asks the question
    if last_speaker is None:
        return groupchat.agent_lookup["Human"]

    # Human → Agent A (explainer)
    if last_speaker.name == "Human":
        return groupchat.agent_lookup["AgentA_Explainer"]

    # Agent A → Agent B (retriever)
    if last_speaker.name == "AgentA_Explainer":
        return groupchat.agent_lookup["AgentB_MaterialsRetriever"]

    # Agent B → Agent C (analyzer)
    if last_speaker.name == "AgentB_MaterialsRetriever":
        return groupchat.agent_lookup["AgentC_Analyzer"]

    # Agent C → Human (deliver final output)
    if last_speaker.name == "AgentC_Analyzer":
        return groupchat.agent_lookup["Human"]

    # Agent D (coder) → Human
    if last_speaker.name == "AgentD_Coder":
        return groupchat.agent_lookup["Human"]

    # Fallback
    return groupchat.agent_lookup["Human"]


# Create the GroupChat
group_chat = GroupChat(
    agents=[
        Human,
        AgentA_Explainer,
        AgentB_MaterialsRetriever,
        AgentC_Analyzer,
        AgentD_Coder,
    ],
    messages=[],
    max_round=20,
    speaker_selection_method=select_next_speaker
)

# Required lookup table for AG2
group_chat.agent_lookup = {agent.name: agent for agent in group_chat.agents}

# Attach context variables
group_chat.context_variables = context_variables

# Manager
manager = GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config
)

print("Pattern loaded.")


Pattern loaded.


# Group Chat

In [ ]:


print("\n=== WildfiresAI | Multi-Agent Materials System ===\n")

# Start the conversation from the Human agent.
# Human ALWAYS starts 
Human.initiate_chat(
    manager,
    message="Please enter your materials-science question.",
    context_variables=context_variables
)




=== WildfiresAI | Multi-Agent Materials System ===

Human (to chat_manager):

Please enter your materials-science question.

--------------------------------------------------------------------------------

Next speaker: AgentA_Explainer

AgentA_Explainer (to chat_manager):

{ "user_query": "What materials can be used to make flexible solar cells and how do their electrical properties enable flexibility?" }

--------------------------------------------------------------------------------

Next speaker: AgentB_MaterialsRetriever

AgentB_MaterialsRetriever (to chat_manager):

{
  "explained_terms": "The query is asking for materials suitable for use in flexible solar cells. Important properties to consider for the search include: - Band gap: crucial for determining a material's ability to absorb sunlight and convert it into electricity. A specific range is typically desired for solar applications. - Elements: certain elements are known for use in flexible solar technologies (e.g., organ